In [155]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
from sklearn import metrics
import os
import yaml
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from matplotlib import pyplot as plt

from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\murat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\murat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
df = pd.read_csv('/Users/murat/OneDrive/Masaüstü/VeriProje/train.csv')

In [157]:
df.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26650,approved,0,0,0,1,0,0.000000,0,4


In [158]:
df.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [159]:
df.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26650,approved,0,0,0,1,0,0.000000,0,4


In [160]:
def processQues(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub('\[\d+\]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

In [161]:
stop_words = set(stopwords.words('english'))

In [162]:
def CleanText(comment):
    comment=processQues(comment)
    comment = nltk.word_tokenize(comment)
    w = [wd for wd in comment if wd not in stop_words]
    return " ".join(w)

In [163]:
df["comment_text"]=df["comment_text"].apply(CleanText)

In [164]:
df["comment_text"] = df['comment_text'].str.replace('[^\w\s]','')

C:\Users\murat\AppData\Local\Temp\ipykernel_18488\513330713.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["comment_text"] = df['comment_text'].str.replace('[^\w\s]','')


In [165]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,cool like would want mother read really grea...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,thank would make life lot less anxietyinducing...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,urgent design problem kudos taking impressive,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,something ill able install site releasing,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha guys bunch losers,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [167]:
def stem_sentences(sentence):
    tokens = sentence.split()
    porter_stemmer = PorterStemmer()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['comment_text'] = df['comment_text'].apply(stem_sentences)

In [168]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,cool like would want mother read realli great ...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,thank would make life lot less anxietyinduc ke...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,urgent design problem kudo take impress,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,someth ill abl instal site releas,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha guy bunch loser,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [169]:
df.isnull().sum(axis = 0)

id                                           0
target                                       0
comment_text                                 0
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                                 1399744
male         

In [170]:
df.dropna()
df = df.dropna(thresh=len(df) - 3, axis=1)

In [171]:
delete_columns = ["rating","created_date", "publication_id","article_id"]

for delete in delete_columns:
    df.drop(delete,axis=1,inplace=True)

In [172]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,cool like would want mother read realli great ...,0.000000,0.0,0.000000,0.00000,0.0,0,0,0,0,0,0.0,0,4
1,59849,0.000000,thank would make life lot less anxietyinduc ke...,0.000000,0.0,0.000000,0.00000,0.0,0,0,0,0,0,0.0,0,4
2,59852,0.000000,urgent design problem kudo take impress,0.000000,0.0,0.000000,0.00000,0.0,0,0,0,0,0,0.0,0,4
3,59855,0.000000,someth ill abl instal site releas,0.000000,0.0,0.000000,0.00000,0.0,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha guy bunch loser,0.021277,0.0,0.021277,0.87234,0.0,0,0,0,1,0,0.0,4,47


In [173]:
df["non_toxic"] = df.iloc[:,3:8].apply(lambda x: 1 if (sum(x)==0) else 0, axis=1)

In [174]:
df["less_toxic"] = df.iloc[:,3:8].apply(lambda x: 1 if (sum(x)/5<=0.2 and sum(x)!=0) else 0, axis=1)

In [175]:
df["very_toxic"] = df.iloc[:,3:8].apply(lambda x: 1 if (sum(x)/5>0.2) else 0, axis=1)

In [176]:
df.head(1000)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,non_toxic,less_toxic,very_toxic
0,59848,0.000000,cool like would want mother read realli great ...,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
1,59849,0.000000,thank would make life lot less anxietyinduc ke...,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
2,59852,0.000000,urgent design problem kudo take impress,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
3,59855,0.000000,someth ill abl instal site releas,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
4,59856,0.893617,haha guy bunch loser,0.021277,0.00000,0.021277,0.872340,0.0,0,0,0,1,0,0.000000,4,47,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,241345,0.000000,call dib tonya hard pic,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,1,0,0.000000,0,4,1,0,0
996,241346,0.629630,freak sad anoth great place lost greedi develo...,0.086420,0.54321,0.049383,0.432099,0.0,0,0,0,2,0,0.012346,0,81,0,0,1
997,241348,0.000000,test comment tri new civil comment platform ar...,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
998,241349,0.000000,test comment tri new civil comment platform ar...,0.000000,0.00000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,4,4,1,0,0


In [184]:
df_sample = df.iloc[1:10000,:]

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [179]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df_sample['comment_text'])

In [182]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [191]:
# data scaling
x_scaled = StandardScaler().fit_transform(df.loc[1:2000, df.columns != 'comment_text'])


In [192]:
from sklearn.decomposition import PCA
 
pca = PCA(n_components=3)
 
pca_features = pca.fit_transform(x_scaled)
 
print('Shape before PCA: ', x_scaled.shape)
print('Shape after PCA: ', pca_features.shape)
pca_df = pd.DataFrame(
    data=pca_features, 
    columns=['PC1', 'PC2', 'PC3'])

Shape before PCA:  (2000, 18)
Shape after PCA:  (2000, 3)


In [193]:
pca_df.head()

,PC1,PC2,PC3
0,-0.332517,2.382158,-2.966680
1,-0.332528,2.382126,-2.966631
2,-0.332539,2.382093,-2.966582
3,7.107798,0.027889,-3.335464
4,9.396333,9.596414,-4.435922


In [194]:
pca.explained_variance_

array([4.57256817, 1.68731539, 1.18376881])

In [195]:
df_sample.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,non_toxic,less_toxic,very_toxic
1,59849,0.000000,thank would make life lot less anxietyinduc ke...,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
2,59852,0.000000,urgent design problem kudo take impress,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
3,59855,0.000000,someth ill abl instal site releas,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,0.000000,0,4,1,0,0
4,59856,0.893617,haha guy bunch loser,0.021277,0.000000,0.021277,0.872340,0.0,0,0,0,1,0,0.000000,4,47,0,1,0
5,59859,0.666667,ur sh tti comment,0.047619,0.638095,0.000000,0.333333,0.0,0,0,0,0,0,0.009524,0,105,0,0,1


In [211]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(df.iloc[1:1000,3:8], df['non_toxic'].iloc[1:1000])

knn.predict(df_sample.iloc[:1,3:8])

C:\Users\murat\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array([1], dtype=int64)